In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime

import hvplot.pandas
import holoviews as hv

import warnings

from datetime import datetime
today = datetime.today().strftime('%m/%d/%Y')

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

states_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'
counties_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'

state_name = 'California'

county_populations = {
    'San Mateo': 765935, 
    'San Francisco': 870044,
    'Santa Clara': 1922200,
    'Alameda': 1643700   
}

interested_counties = list(county_populations.keys())


hv.config(future_deprecations=True)

hv.notebook_extension('bokeh')


In [ ]:
def prepare_cases(cases, rolling_window=7):
    new_cases = cases.diff()

    smoothed = new_cases.rolling(
        rolling_window,
        win_type='gaussian',
        min_periods=1,
        center=True).mean(std=2).round()
    
    zeros = smoothed.index[smoothed.eq(0)]
    if len(zeros) == 0:
        idx_start = 0
    else:
        last_zero = zeros.max()
        idx_start = smoothed.index.get_loc(last_zero) + 1
        
    smoothed = smoothed.iloc[idx_start:]
    original = new_cases.loc[smoothed.index]
    
    return original, smoothed


In [ ]:

states = pd.read_csv(states_url,
                     usecols=[0,1,3], # 3 is new cases
                     index_col=['state', 'date'],
                     parse_dates=['date']).sort_index()
    
original, smoothed = prepare_cases(states.xs(state_name).squeeze())

pd.DataFrame({
        "Actual": original, 
        "7 Day Rolling": smoothed
    }).hvplot(
        title = f"{state_name} Daily New Cases [{today}]", 
        grid=True,
        color=hv.Cycle(['lightgrey', 'red'])
)


In [ ]:

states = pd.read_csv(states_url,
                     usecols=[0,1,4], # 4 is deaths
                     index_col=['state', 'date'],
                     parse_dates=['date']).sort_index()

original, smoothed = prepare_cases(states.xs(state_name).squeeze())

pd.DataFrame({
        "Actual": original, 
        "7 Day Rolling": smoothed
    }).hvplot(
        title = f"{state_name} Daily Deaths [{today}]", 
        grid=True,
        color=hv.Cycle(['lightgrey', 'red'])
)


In [ ]:
# Bay Area new daily deaths 

counties = pd.read_csv(counties_url,
              parse_dates=['date'],
              squeeze=True)

counties = counties[counties.county.isin(['San Mateo'])]\
        .drop(columns=['state', 'fips', 'cases'])\
        .set_index(['county', 'date'])\
        .sort_index()

original, smoothed = prepare_cases(counties.xs('San Mateo').squeeze(), 7)

pd.DataFrame({
        "Actual": original, 
        "7 Day Rolling": smoothed
    }).hvplot(
        title = f"Bay Area Daily Deaths [{today}]", 
        grid=True,
        color=hv.Cycle(['lightgrey', 'red'])
)


In [ ]:
# new cases in 4 counties broken out


counties = pd.read_csv(counties_url,
              parse_dates=['date'],
              squeeze=True)

counties = counties[counties.county.isin(interested_counties)]\
        .drop(columns=['state', 'fips', 'deaths'])\
        .set_index(['county', 'date'])\
        .sort_index()

mergedDf = pd.DataFrame()

for i, county_name in enumerate(interested_counties):

    _, smoothed = prepare_cases(counties.xs(county_name).squeeze())
    
    mergedDf[county_name] = smoothed
    
mergedDf['Bay Area'] = mergedDf[interested_counties].sum(axis=1)

mergedDf.hvplot(
    title=f"Daily New Cases by County [{today}]",
    color=hv.Cycle(['#A904BF', '#3B0273', '#EBF20C', '#F2B90C', 'red']),
    grid=True)
